In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import nibabel as nib
import pandas as pd
from torchvision import transforms
from tqdm import tqdm
import os
import datetime
import pandas as pd
import numpy as np
import nibabel as nib
import torch
from torch.utils.data import Dataset

In [2]:
import importlib.util
import sys

# Specify the full path to the module file
module_path = 'D:\\Github Folder\\MasterThesis\\notebooks\\FUNCTIONS.py'

# Load the module
spec = importlib.util.spec_from_file_location("FUNCTIONS", module_path)
functions = importlib.util.module_from_spec(spec)
spec.loader.exec_module(functions)

# Now you can use the functions as if you had imported them
load_datasets = functions.load_datasets
create_dataloaders = functions.create_dataloaders
train_and_validate = functions.train_and_validate
test_model = functions.test_model


In [3]:
df = pd.read_excel("references\Subject_info_balanced.xlsx")

In [4]:
# Assuming 'df' is your DataFrame loaded with the 'Research Group' column available
label_categories = pd.Categorical(df['Research Group'])
label_mapping = {code: category for code, category in enumerate(label_categories.categories)}

In [8]:
class Baseline3DCNN(nn.Module):
    def __init__(self, num_classes=2, init_filters=8, kernel_size=2, stride=2, num_fc_units=128):
        super(Baseline3DCNN, self).__init__()
        self.init_filters = init_filters
        self.kernel_size = kernel_size
        self.stride = stride
        self.num_fc_units = num_fc_units

        self.conv1 = nn.Conv3d(1, init_filters, kernel_size=kernel_size, stride=stride, padding=kernel_size//2)
        self.conv2 = nn.Conv3d(init_filters, init_filters*2, kernel_size=kernel_size, stride=stride, padding=kernel_size//2)
        self.conv3 = nn.Conv3d(init_filters*2, init_filters*4, kernel_size=kernel_size, stride=stride, padding=kernel_size//2)
        self.pool = nn.MaxPool3d(2)
        self.relu = nn.ReLU()

        # Example input to determine the size
        example_input = torch.randn(1, 1, 193, 229, 193)
        example_output = self.forward_features(example_input)
        self.final_dim = example_output.numel()

        self.fc1 = nn.Linear(self.final_dim, num_fc_units)
        self.fc2 = nn.Linear(num_fc_units, num_classes)

    def forward_features(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        return torch.flatten(x, start_dim=1)

    def forward(self, x):
        x = self.forward_features(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [9]:
import os
import datetime
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from openpyxl import load_workbook

# Assuming Baseline3DCNN, train_and_validate, test_model, and related utilities are defined elsewhere

def run_experiment(df, config):
    """Run the experiment with the given configuration on the preprocessed DataFrame."""
    train_dataset, val_dataset, test_dataset = load_datasets(df, config['image_type'], sample_size=config['sample_size'])
    train_loader, val_loader, test_loader = create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=config['batch_size'])
    
    # Initialize model and training components
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Baseline3DCNN(num_classes=config['num_classes'], init_filters=config['init_filters'],
                          kernel_size=config['kernel_size'], stride=config['stride'], num_fc_units=config['num_fc_units']).to(device)
    

    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    # Training and validation
    train_accuracies, val_accuracies, val_losses = train_and_validate(model, train_loader, val_loader, criterion, optimizer, label_mapping, config['num_epochs'], config['patience'], device)
    test_results, test_accuracy = test_model(model, test_loader, label_mapping, device)
    
    # Save detailed results to Excel
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime('%Y-%m-%d_%H-%M-%S')
    report_filename = os.path.join('reports', f'{formatted_time}_Experiment.xlsx')
    onnx_filename = os.path.join('models', f'{formatted_time}_Model.onnx')
    
    summary_data = {
        'Phase': ['Training', 'Validation', 'Testing'],
        'Accuracy': [train_accuracies[-1], val_accuracies[-1], test_accuracy]
    }
    summary_df = pd.DataFrame(summary_data)
    all_results = pd.DataFrame(test_results)
    config_df = pd.DataFrame([config])
    
    with pd.ExcelWriter(report_filename) as writer:
        config_df.to_excel(writer, sheet_name='Configuration')
        all_results.to_excel(writer, sheet_name='Results')
        summary_df.to_excel(writer, sheet_name='Summary')

    # Append a summary of this experiment to the cumulative RESULTS.xlsx file
    results_file = os.path.join('reports', 'RESULTS.xlsx')
    experiment_summary = {**config, **{'Training Accuracy': train_accuracies[-1], 'Validation Accuracy': val_accuracies[-1], 'Test Accuracy': test_accuracy, 'DATETIME': formatted_time}}
    summary_row = pd.DataFrame([experiment_summary])

    if os.path.exists(results_file):
        with pd.ExcelWriter(results_file, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            existing_df = pd.read_excel(results_file)
            combined_df = pd.concat([existing_df, summary_row], ignore_index=True)
            combined_df = combined_df.reindex(columns=(existing_df.columns.tolist() + [col for col in summary_row.columns if col not in existing_df.columns]))
            combined_df.to_excel(writer, index=False, sheet_name='Sheet1')
    else:
        summary_row.to_excel(results_file, index=False)

    # Save the model to ONNX
    dummy_input = torch.randn(1, 1, 193, 229, 193, device=device)  # Adjust size according to your model's input
    torch.onnx.export(model, dummy_input, onnx_filename, export_params=True)

    return report_filename, train_accuracies[-1], val_accuracies[-1], test_accuracy


In [11]:
image_types = ['PET']
init_filters_options = [32, 64, 128, 256]
kernel_size_options = [2]
batch_size_options = [ 4, 5, 6]

# Base configuration for experiments
base_config = {
    'num_classes': 2,
    'stride': 2,
    'num_fc_units': 128,
    'optimizer': 'Adam',
    'loss_criterion': 'CrossEntropyLoss',
    'num_epochs': 20,
    'patience': 3,
    'Description': 'Baseline3DCNN - Simple model architecture',
    'sample_size': None
}

results = []

# Outermost loop for image types
for image_type in image_types:
    # Update the image type in the configuration
    config = base_config.copy()
    config['image_type'] = image_type
    
    print(f"Working on image type: {image_type}")
    
    # Loop for init_filters
    for init_filter in init_filters_options:
        config['init_filters'] = init_filter
        
        # Loop for kernel_size
        for kernel_size in kernel_size_options:
            config['kernel_size'] = kernel_size
            
            # Loop for batch_size
            for batch_size in batch_size_options:
                config['batch_size'] = batch_size
                
                # Log the current configuration
                print(f"Running experiment with init_filters={init_filter}, kernel_size={kernel_size}, batch_size={batch_size}")
                
                # Run the experiment and store the result
                result = run_experiment(df, config)
                results.append((config.copy(), result))

# Print or process results as needed
print("Experiments completed. Results collected.")


Working on image type: PET
Running experiment with init_filters=32, kernel_size=2, batch_size=4


Epoch 1/20 - Train: 100%|██████████| 19/19 [03:57<00:00, 12.49s/it]


Epoch 1: Train Loss: 0.7740 - Train Accuracy: 44.74%


Epoch 1/20 - Validate: 100%|██████████| 6/6 [00:59<00:00,  9.98s/it]


Epoch 1: Validation Loss: 0.6952 - Validation Accuracy: 45.83%


...

Epoch 6: Validation Loss: 0.7516 - Validation Accuracy: 45.83%
Early stopping triggered after 6 epochs due to no improvement in validation loss or accuracy.


Testing: 100%|██████████| 6/6 [00:56<00:00,  9.48s/it]
C:\Users\Micha\AppData\Local\Temp\ipykernel_42940\3617251816.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([existing_df, summary_row], ignore_index=True)


Running experiment with init_filters=32, kernel_size=2, batch_size=5


Epoch 1/20 - Train:   0%|          | 0/15 [00:18<?, ?it/s]


KeyboardInterrupt: 